In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from architectures.backbones import MobileNet

In [6]:
model = MobileNet.mobilenet_v2(pretrained=True)

In [10]:
import torch
a = torch.rand(1,3,512,512)

# default os=32, expect 512//32 with 1280 channels output
b = model(a)
print(b.shape)

torch.Size([1, 1280, 16, 16])


In [13]:
total_params = sum(p.numel() for p in model.parameters())
print(total_params)

2223872


In [16]:
model_half_width = MobileNet.mobilenet_v2(width_mult=0.5)
total_params = sum(p.numel() for p in model_half_width.parameters())
print(total_params)

687680
